# Cryptopunks: obtener datos de un cryptopunk

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

import mysql.connector
import random

import re

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Función: Obtener la información del cryptopunk

In [3]:
def obtener_cryptopunk(soup, mydb):
    # Obtener el tipo de cryptopunk
    h4 = soup.find_all('h4')
    tipo_cp = h4[0].text.split()[3]
    print(tipo_cp)
    
    # Obtener el nombre del owner, y saber si está a la venta y su precio
    varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

    for linea in varios:
        texto = linea.text
        # Busco la línea que comienza con ...
        if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
            # Extraigo el nombre del dueño
            if texto.find("This punk is currently owned by address")>0:
                owner = texto.split('address ')[1].split('.',1)[0]
                print('Owner:',owner)
        
            # Averiguo si está en venta
            precio_venta = 0
            if texto.find("This punk is currently for sale by owner")>0:
                sale = True
                precio_venta = texto.split('by owner for ')[1].split('Ξ',1)[0]
                print('Precio de venta', precio_venta)
            else:
                sale = False # Dice 'This punk has not been listed for sale by its owner'
            print('For sale',sale)
        
            # Averiguo si hay ofertas
            oferta = 0
            if texto.find("There is a bid of")>0:
                bids = True
                oferta = texto.split('is a bid of ')[1].split('Ξ',1)[0]
                print('Valor ofrecido', oferta)
             
            else:
                bids = False  # Dice "There are currently no bids on this punk."
            print('Bids:',bids)

    # Obtain the typeID
    mycursor.execute("SELECT id FROM ctypes WHERE name = %s", (tipo_cp.strip(),))
    id_tipo = mycursor.fetchone()[0]

    # Insert cryptopunk data into the table
    sql = "INSERT INTO cryptopunks (id, owner, id_type, for_sale, bids) VALUES (%s, %s, %s, %s, %s )"

    mycursor.execute(sql, (nro_cp, owner, id_tipo, sale, bids ))

    
    mydb.commit()        

## Función: Obtener la lista de transacciones realizadas

In [4]:
def obtener_transacciones(soup, mydb):
    
    def obtener_montos(str1):
        try:
            salida = re.search("(((\(\$)\s*\d+)([,]\d+){0,1}([,]\d+){0,1}([.]?\d{1,2})?)[MK]?\)", str1)
            con_coma = salida[0][2:-1]
            sin_coma = con_coma.replace(',','')
            if sin_coma.find('M')>0:
                sin_coma = sin_coma[0:-1]
                monto_dols = float(sin_coma)
                monto_dols = monto_dols * 1000000
            elif sin_coma.find('K')>0:
                sin_coma = sin_coma[0:-1]
                monto_dols = float(sin_coma)
                monto_dols = monto_dols * 1000
            else: 
                monto_dols = float(sin_coma)
            con_E = re.search("(\d+([,]\d+)?([.]\d{2})?)?\Ξ", str1)
            monto = float(con_E[1].replace(',',''))
            return monto, monto_dols
        except:
            return 0,0
        
    text1 = soup.find_all('a', text = '')
    #print(text1)
    
    transacciones =[]
    
    transac =soup.find('table', class_="table")
    #print(transac)

    for tr in transac.find_all('tr'):
        lista = []
        for td in tr.find_all('td'):
            lista.append(td.text.strip())
        transacciones.append(lista)
    transacciones = transacciones[1:]
    print(transacciones)
    
  
    # Insert data into the table
    sql = "INSERT INTO transactions (id_cryptopunk, tr_type, tr_from, tr_to, amount, amount_dols, txn) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for tr in transacciones:
        # si la transaccion no esta vacia
        if len(tr)>0:
            # Hay un monto de transacción?
            monto = 0
            monto_dols = 0
            if len(tr[3])>0:
                monto, monto_dols = obtener_montos(tr[3])
            
            fecha = datetime.strptime(tr[4], "%b %d, %Y")
            
    
            mycursor.execute(sql, (nro_cp, tr[0],  tr[1], tr[2], monto, monto_dols, fecha))

    
    mydb.commit()

## Función: Obtener atributos del cryptopunk

In [5]:
def obtener_atributos(soup, mydb):
    cod =soup.find_all('div', class_="col-md-4")
    textos = []
    for c in cod:
        textos.append(c.find('a').text)
    
    atributos = textos[1:]
    print('Atributos:', atributos)
    
    # Insert data into the table attributes
    sql = "INSERT INTO cryptopunk_has_attributes (id_cryptopunk, id_attribute) VALUES (%s, %s)"
    

    for atr in atributos:
        # Obtain the typeID
        mycursor.execute("SELECT id FROM attributes WHERE name = %s", (atr.strip(),))
        id_atr = mycursor.fetchone()[0]
    
        mycursor.execute(sql, (nro_cp,id_atr))


    mydb.commit()

## Ejecución del proceso

In [ ]:
# Rango de cryptopunks a scrapear
for nro_cp in range(5697,10000):
    
    # Define cryptopunk to scrap
    url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro_cp)

    # Use BeautifulSoup por webscrapping
    html=requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')

    obtener_cryptopunk(soup, mydb)
    obtener_transacciones(soup, mydb)
    obtener_atributos(soup, mydb)
    time.sleep(random.uniform(5.0, 8.0))

Female
Owner: 0x030def
For sale False
Bids: False
[['Transfer', '0xc272ed', '0x030def', '', 'Sep 12, 2021'], ['Bid Withdrawn', 'TweetyPi…', '', '0.10Ξ ($13)', 'Dec 29, 2019'], ['Bid', 'TweetyPi…', '', '0.10Ξ ($12)', 'Dec 28, 2019'], ['Bid Withdrawn', 'Dude_Nak…', '', '0.12Ξ ($86)', 'Mar 11, 2018'], ['Bid', 'Dude_Nak…', '', '0.12Ξ ($87)', 'Mar 09, 2018'], ['Claimed', '', '0xc272ed', '', 'Jun 23, 2017']]
Atributos: ['Black Lipstick', 'Crazy Hair', 'Nerd Glasses']
Female
Owner: 0x35126e
For sale False
Bids: False
[['Transfer', '0x0452b1', '0x35126e', '', 'Jul 29, 2020'], ['Sold', 'Hemba', '0x0452b1', '0.95Ξ ($125)', 'Mar 21, 2020'], ['Offered', '', '', '0.95Ξ ($248)', 'Feb 22, 2020'], ['Claimed', '', 'Hemba', '', 'Jun 23, 2017']]
Atributos: ['Purple Eye Shadow', 'Headband', 'Earring']
Male
Owner: 0xcffc33
For sale False
Bids: False
[['Transfer', '0xba7ccc', '0xcffc33', '', 'May 05, 2021'], ['Claimed', '', '0xba7ccc', '', 'Jun 23, 2017']]
Atributos: ['Vampire Hair', 'Cigarette', 'Normal Be

Male
Owner: 0x560b92
For sale False
Bids: False
[['Bid Withdrawn', 'hotbtch', '', '0.52Ξ ($107)', 'Jul 29, 2017'], ['Sold', '0xc00c2b', '0x560b92', '0.64Ξ ($122)', 'Jul 11, 2017'], ['Offered', '', '', '0.63Ξ ($169)', 'Jul 06, 2017'], ['Offer Withdrawn', '', '', '', 'Jul 05, 2017'], ['Bid', 'hotbtch', '', '0.52Ξ ($138)', 'Jul 05, 2017'], ['Offered', '', '', '0.62Ξ ($167)', 'Jul 04, 2017'], ['Offered', '', '', '0.43Ξ ($112)', 'Jul 01, 2017'], ['Bid', '0x7760e0', '', '0.04Ξ ($12)', 'Jun 28, 2017'], ['Offered', '', '', '0.30Ξ ($86)', 'Jun 27, 2017'], ['Claimed', '', '0xc00c2b', '', 'Jun 23, 2017']]
Atributos: ['Messy Hair', 'Big Shades', 'Shadow Beard', 'Earring']
Female
Owner: ross_VRO…
For sale False
Bids: False
[['Bid Withdrawn', 'tokyolif…', '', '80Ξ ($246,372)', 'May 03, 2021'], ['Bid', 'tokyolif…', '', '80Ξ ($244,481)', 'May 03, 2021'], ['Bid Withdrawn', 'anonymou…', '', '45Ξ ($102,442)', 'Apr 24, 2021'], ['Bid', 'anonymou…', '', '45Ξ ($103,878)', 'Apr 23, 2021'], ['Sold', '0x577ebc'

Male
Owner: 0x231d68
For sale False
Bids: False
[['Sold', '0xe281b9', '0x231d68', '47.50Ξ ($131,904)', 'Aug 06, 2021'], ['Offered', '', '', '47.50Ξ ($132,448)', 'Aug 05, 2021'], ['Offered', '', '', '46Ξ ($119,760)', 'Aug 05, 2021'], ['Offered', '', '', '40Ξ ($99,634)', 'Aug 03, 2021'], ['Offer Withdrawn', '', '', '', 'May 03, 2021'], ['Offered', '', '', '25.38Ξ ($52,345)', 'Apr 09, 2021'], ['Sold', '0xa6e2e9', '0xe281b9', '21.97Ξ ($38,448)', 'Mar 17, 2021'], ['Offered', '', '', '21.97Ξ ($38,681)', 'Mar 17, 2021'], ['Bid Withdrawn', '✨🚀🌙.et…', '', '15Ξ ($23,267)', 'Mar 03, 2021'], ['Bid', '✨🚀🌙.et…', '', '15Ξ ($22,226)', 'Feb 26, 2021'], ['Offer Withdrawn', '', '', '', 'Feb 26, 2021'], ['Offered', '', '', '21.50Ξ ($31,292)', 'Feb 26, 2021'], ['Claimed', '', '0xa6e2e9', '', 'Jun 23, 2017']]
Atributos: ['Clown Eyes Blue', 'Mustache', 'Mohawk']
Female
Owner: 0x2cc123
For sale False
Valor ofrecido 0.30
Bids: True
[['Transfer', 'Snowfro', '0x2cc123', '', 'Jul 31, 2021'], ['Sold', 'jmg', 'Snow

Male
Owner: 0xa002c3
For sale False
Bids: False
[['Transfer', '0xa397d0', '0xa002c3', '', 'Dec 31, 2021'], ['Offered', '', '', '501Ξ ($2.05M)', 'Dec 27, 2021'], ['Offer Withdrawn', '', '', '', 'Dec 23, 2021'], ['Offered', '', '', '250Ξ ($1M)', 'Dec 19, 2021'], ['Sold', 'h4x.eth', '0xa397d0', '199.79Ξ ($801,795)', 'Dec 19, 2021'], ['Bid', '0xa397d0', '', '199.79Ξ ($801,795)', 'Dec 19, 2021'], ['Bid', 'cgpunks.…', '', '199.69Ξ ($801,394)', 'Dec 19, 2021'], ['Bid', '0xa397d0', '', '196.69Ξ ($789,354)', 'Dec 19, 2021'], ['Bid', 'cgpunks.…', '', '190.70Ξ ($762,865)', 'Dec 19, 2021'], ['Offer Withdrawn', '', '', '', 'Dec 19, 2021'], ['Bid', '0xa397d0', '', '190.69Ξ ($762,825)', 'Dec 19, 2021'], ['Bid', 'yfimaxi.…', '', '160Ξ ($640,054)', 'Dec 19, 2021'], ['Offered', '', '', '199.69Ξ ($794,922)', 'Dec 19, 2021'], ['Offered', '', '', '250Ξ ($983,275)', 'Dec 19, 2021'], ['Offered', '', '', '300Ξ ($1.2M)', 'Dec 15, 2021'], ['Offered', '', '', '420Ξ ($1.66M)', 'Dec 10, 2021'], ['Offer Withdrawn',

Male
Owner: 0xe9562d
For sale False
Bids: False
[['Sold', 'punksotc…', '0xe9562d', '96Ξ ($301,592)', 'Aug 27, 2021'], ['Offered', '', '', '96Ξ ($303,555)', 'Aug 27, 2021'], ['Offered', '', '', '91.10Ξ ($283,932)', 'Aug 26, 2021'], ['Offered', '', '', '94Ξ ($301,926)', 'Aug 25, 2021'], ['Sold', '0xb7497b', 'punksotc…', '81Ξ ($255,355)', 'Aug 25, 2021'], ['Offered', '', '', '81Ξ ($254,143)', 'Aug 25, 2021'], ['Offer Withdrawn', '', '', '', 'Aug 25, 2021'], ['Offered', '', '', '85Ξ ($270,812)', 'Aug 25, 2021'], ['Sold', 'rockinro…', '0xb7497b', '69.69Ξ ($232,305)', 'Aug 23, 2021'], ['Offered', '', '', '69.69Ξ ($231,005)', 'Aug 15, 2021'], ['Offered', '', '', '79.69Ξ ($246,366)', 'Aug 12, 2021'], ['Bid Withdrawn', 'spritzlo…', '', '55Ξ ($174,672)', 'Aug 12, 2021'], ['Sold', '0x2614e7', 'rockinro…', '57Ξ ($181,024)', 'Aug 11, 2021'], ['Bid', 'spritzlo…', '', '55Ξ ($174,672)', 'Aug 11, 2021'], ['Bid Withdrawn', 'spritzlo…', '', '52.50Ξ ($166,733)', 'Aug 11, 2021'], ['Offered', '', '', '57Ξ (

...

Male
Owner: 0xedad6b
For sale False
Bids: False
[['Transfer', '0x4680cb', '0xedad6b', '', 'Dec 13, 2021'], ['Offered', '', '', '249Ξ ($1.08M)', 'Nov 21, 2021'], ['Offered', '', '', '269.69Ξ ($1.21M)', 'Nov 02, 2021'], ['Transfer', '0x78b58e', '0x4680cb', '', 'Oct 30, 2021'], ['Bid Withdrawn', 'beautifu…', '', '190Ξ ($761,925)', 'Oct 28, 2021'], ['Bid', 'beautifu…', '', '190Ξ ($757,916)', 'Oct 28, 2021'], ['Offered', '', '', '269.69Ξ ($1.08M)', 'Oct 27, 2021'], ['Offered', '', '', '350Ξ ($1.48M)', 'Oct 26, 2021'], ['Transfer', '0xe9c636', '0x78b58e', '', 'Oct 22, 2021'], ['Transfer', 'peteschi…', '0xe9c636', '', 'Aug 29, 2021'], ['Offer Withdrawn', '', '', '', 'Aug 29, 2021'], ['Offered', '', '', '666Ξ ($1.83M)', 'Aug 06, 2021'], ['Offered', '', '', '420.69Ξ ($818,537)', 'Jul 21, 2021'], ['Offered', '', '', '200Ξ ($400,152)', 'Jun 23, 2021'], ['Transfer', '0xdc25e8', 'peteschi…', '', 'Jun 23, 2021'], ['Bid Withdrawn', 'beautifu…', '', '49Ξ ($107,285)', 'Jun 19, 2021'], ['Bid', 'beautifu

Male
Owner: 0xa66d56
For sale False
Bids: False
[['Bid Withdrawn', 'punksotc…', '', '96Ξ ($365,388)', 'Jan 03, 2022'], ['Sold', 'squid42.…', '0xa66d56', '109.90Ξ ($418,294)', 'Jan 03, 2022'], ['Bid', 'punksotc…', '', '96Ξ ($367,349)', 'Jan 02, 2022'], ['Bid', 'ethblowe…', '', '95Ξ ($363,522)', 'Jan 02, 2022'], ['Bid', 'punksotc…', '', '90Ξ ($344,165)', 'Jan 02, 2022'], ['Offered', '', '', '109.90Ξ ($420,264)', 'Jan 02, 2022'], ['Offered', '', '', '119Ξ ($452,465)', 'Jan 02, 2022'], ['Offered', '', '', '124.75Ξ ($467,754)', 'Jan 02, 2022'], ['Sold', 'pablopun…', 'squid42.…', '180Ξ ($635,368)', 'Oct 06, 2021'], ['Offered', '', 'squid42.…', '180Ξ ($635,368)', 'Oct 06, 2021'], ['Bid Withdrawn', 'squid42.…', '', '172.72Ξ ($607,619)', 'Oct 06, 2021'], ['Offer Withdrawn', '', '', '', 'Oct 06, 2021'], ['Bid', 'squid42.…', '', '172.72Ξ ($607,619)', 'Oct 05, 2021'], ['Bid Withdrawn', 'squid42.…', '', '169.69Ξ ($596,959)', 'Oct 05, 2021'], ['Offered', '', '', '187Ξ ($657,855)', 'Oct 05, 2021'], [

Female
Owner: somewher…
Precio de venta 420
For sale True
Bids: False
[['Offered', '', '', '420Ξ ($1.36M)', 'Aug 28, 2021'], ['Sold', '0x131b2b', 'somewher…', '190Ξ ($615,507)', 'Aug 28, 2021'], ['Offered', '', '', '190Ξ ($634,693)', 'Aug 23, 2021'], ['Offered', '', '', '170Ξ ($471,396)', 'Aug 06, 2021'], ['Sold', '0xb70c1a', '0x131b2b', '104.99Ξ ($288,293)', 'Aug 06, 2021'], ['Offered', '', '', '104.99Ξ ($290,115)', 'Aug 06, 2021'], ['Offered', '', '', '99.99Ξ ($276,298)', 'Aug 06, 2021'], ['Offered', '', '', '94.99Ξ ($266,885)', 'Aug 05, 2021'], ['Offered', '', '', '89.99Ξ ($244,191)', 'Aug 04, 2021'], ['Offered', '', '', '84.99Ξ ($230,708)', 'Aug 04, 2021'], ['Offered', '', '', '79.99Ξ ($201,016)', 'Aug 04, 2021'], ['Offered', '', '', '78.99Ξ ($200,942)', 'Aug 03, 2021'], ['Offered', '', '', '80Ξ ($202,526)', 'Jul 31, 2021'], ['Sold', '0x062c54', '0xb70c1a', '69.99Ξ ($177,185)', 'Jul 31, 2021'], ['Offered', '', '', '69.99Ξ ($149,796)', 'Apr 18, 2021'], ['Bid Withdrawn', 'daddykal…',

Male
Owner: 0xc5cf9f
Precio de venta 100
For sale True
Bids: False
[['Offered', '', '', '100Ξ ($247,897)', 'Jan 25, 2022'], ['Offer Withdrawn', '', '', '', 'Jan 25, 2022'], ['Offered', '', '', '85Ξ ($271,844)', 'Jan 09, 2022'], ['Transfer', '0xc5016d', '0xc5cf9f', '', 'Jan 08, 2022'], ['Transfer', 'bagerart…', '0xc5016d', '', 'Jan 07, 2022'], ['Offered', '', '', '86.50Ξ ($354,439)', 'Dec 24, 2021'], ['Offer Withdrawn', '', '', '', 'Dec 24, 2021'], ['Offered', '', '', '70Ξ ($287,223)', 'Dec 24, 2021'], ['Sold', '0xdbd994', 'bagerart…', '53Ξ ($214,359)', 'Dec 22, 2021'], ['Bid', 'bagerart…', '', '53Ξ ($214,359)', 'Dec 22, 2021'], ['Bid Withdrawn', 'bagerart…', '', '53Ξ ($214,832)', 'Dec 22, 2021'], ['Bid', 'bagerart…', '', '53Ξ ($214,832)', 'Dec 22, 2021'], ['Bid', '0x72014b', '', '51Ξ ($205,473)', 'Dec 22, 2021'], ['Bid Withdrawn', 'punksotc…', '', '50.50Ξ ($202,928)', 'Dec 22, 2021'], ['Bid', 'punksotc…', '', '50.50Ξ ($203,634)', 'Dec 21, 2021'], ['Offered', '', '', '56Ξ ($223,448)', '

Male
Owner: 0x3be134
For sale False
Bids: False
[['Transfer', '0x4682aa', '0x3be134', '', 'Sep 12, 2021'], ['Sold', '0x36a5bc', '0x4682aa', '25Ξ ($46,379)', 'Feb 22, 2021'], ['Offered', '', '', '25Ξ ($4,933)', 'Apr 28, 2020'], ['Claimed', '', '0x36a5bc', '', 'Jun 23, 2017']]
Atributos: ['Bandana', 'Chinstrap', 'Nerd Glasses']
